# 주제 : 고객 세분화를 통한 혁신적 기업성장 솔루션 
## 데이터 분석 목표 설정 및 개요
1. 데이터 분석 목표
   +    1. 고객 세분화를 통해 차년도 매출향상 전략 탐색
   +    2. 성장 기반이 되는 잠재 요소 탐색
2. 개요
   + RFM 지표를 활용하여 K-Means Clustering을 진행하고 5가지 세그먼트로 고객을 분류한다.
   + 분류된 세그먼트를 다른 데이터와 비교하여 기업 성장에 필요한 요소를 탐색한다.
   + 분석을 바탕으로 기업성장 솔루션을 제시한다.

## \<목차\>
1. 세팅
   1. 필요 라이브러리 import
   2. 데이터 로드
2. EDA
   1. 데이터 기본 정보 확인(type, shape, null counts, uniques, etc..)
   2. 분석 데이터 통합을 위한 전처리
3. 데이터 분석(시각화)
   + 월별 매출/거래량 추이 분석
   + 월별 마케팅비용 추이 분석
   + 마케팅 비용과 매출과의 선형관계 분석
   + 가입기간과 매출 관계 분석
   + 가입연도별 신규고객동향 파악
   + 지역별 매출/거래량 분석
   + 제품카테고리별 매출/거래량 분석
4. 코호트 분석
   + 월별 코호트 분석
   + 잔존 빈도 분석
   + 잔존율 분석
   + 월별 매출액 리텐션 분석
5. RFM과 K-Means를 활용한 고객 세그먼트 정의    
   + RFM 계산
   + RFM 3D 그래프
   + 이상치를 제거한 고객의 RFM 3차원 그래프
   + RFM 값을 변수로 하는 K-Means 클러스터링 수행
   + QQ-Plot을 통한 정규성 검정
   + K-Means 하이퍼파라미터 K(엘보우, 실루엣) 서칭 & 클러스터링 
   + 클러스터링된 데이터의 RFM 3차원 그래프 분석  
   
6. 클러스터링된 데이터 분석
   + RFM 분석
   + 월 거래량 & 매출 분석
   + 고객군별 제품 카테고리 분석
   + 월별 제품 카테고리 분석
   + ARPU 분석
   + 고객군별 지역 분석
   + 고객군별 성별 분석
   + Q. 마케팅비용이 고객군별 매출/거래량에 어떤 영향을 끼칠까?
   + 쿠폰 분석
7. 결론

# 코드 시작

## 1. 기초 세팅
### 1-1. 필요 라이브러리 import

In [ ]:
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', 120)
pd.set_option('display.max_columns', 40)
pd.set_option('display.max_colwidth', 100)

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.cm as cm # color map

plt.style.use('ggplot')
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

import datetime
from datetime import datetime
import calendar

from scipy import stats
from scipy.stats import skew, norm, probplot, boxcox, kurtosis

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, silhouette_score

import math


### 1-2. 데이터 로드
+ Onlinesales_info: 2019년도 한해 동안 발생한 온라인 거래와 관련된 정보보
+ Customer_info: 고객 관련 정보
+ Discount_info: 할인 관련 정보
+ Marketing_info: 마케팅 비용 관련 정보
+ Tax_info: 세금 관련 정보보

In [ ]:
online_sales = pd.read_csv('data/Onlinesales_info.csv')
discount_coupon = pd.read_csv('data/Discount_info.csv')
customer_data = pd.read_csv('data/customer_info.csv')
marketing_spend = pd.read_csv('data/marketing_info.csv')
tax_amount = pd.read_csv('data/Tax_info.csv')

## 2. EDA
### 2-1. 데이터 기본 정보 확인

+ 타입 확인
+ 갯수 확인
+ 중복값 확인
+ 결측치 확인
+ 결측치 비율 확인
+ 고유값 확인


### 2-2. 데이터 통합을 위한 전처리

In [ ]:
def str_summary(df, pred=None):
    obs = df.shape[0]
    types = df.dtypes
    counts = df.apply(lambda x: x.count())
    uniques = df.apply(lambda x: [x.unique()]).T.squeeze() # 각 feature의 고유값을 시리즈 타입으로 반환
    nulls = df.apply(lambda x: x.isnull().sum())
    distincts = df.apply(lambda x: x.unique().shape[0]) # 각 feature의 고유값 개수
    missing_ratio = (df.isnull().sum() / obs) * 100
    
    print("Data shape: ", df.shape)

    cols = ['Types', 'Counts', 'Distincts', 'Nulls', 'Missing_ratio', 'Uniques']
    structure = pd.concat([types, counts, distincts, nulls, missing_ratio, uniques], axis=1, sort=True)

    structure.columns = cols

    print("================================================")
    print("Data types: ")
    print(structure['Types'].value_counts())
    print("================================================")
    print("\n\n")

    return structure

In [ ]:
data_summary_1 = str_summary(online_sales)
data_summary_2 = str_summary(discount_coupon)
data_summary_3 = str_summary(customer_data)
data_summary_4 = str_summary(marketing_spend)
data_summary_5 = str_summary(tax_amount)

In [ ]:
display(data_summary_1)
display(data_summary_2)
display(data_summary_3)
display(data_summary_4)
display(data_summary_5)

5개 데이터셋 모두 결측치는 없다.


## 3. 데이터 분석 (시각화)
### 3-1. 월별 매출/거래량 추이 분석

### 3-2. 월별 마케팅 비용 추이 분석

### 3-3. 마케팅비용과 매출과의 선형관계 분석

### 3-4. 가입기간과 매출과의 관계

### 3-5. 가입연도별 신규고객동향 파악

### 3-6. 지역별 매출/거래량 분석

### 3-7. 제품카테고리별 매출/거래량 분석

## 4. 코호트 분석

### 4-1. 월별 코호트 분석

### 4-2 잔존 빈도 구하기

### 4-3. 잔존율 분석 

### 4-4. 월별 매출액 리텐션 분석

## 5. RFM & k-means를 활용한 고객 세그먼트 정의
+ 데이터의 RFM값과 K-Means 알고리즘을 활용하여 고객을 세분화한다.

### 5-1. RFM 계산

### 5-2. RFM 3D 그래프 분석

### 5-3. 이상치를 제거한 고객의 RFM 3차원 그래프

### 5-4. RFM 값으로 K-Means 클러스터링 수행
+ _
+ _ 
+ _

### 5-5. QQ-Plot을 통한 정규성 검정

### 5-6. K-Means 하이퍼파라미터 K 서칭 & 클러스터링 (엘보우, 실루엣)

### 5-7 클러스터링 된 데이터의 RFM 3차원 그래프

## 6. 클러스터링 된 데이터 분석
### 6-1. RFM 분석

### 6-2. 월 거래량&매출 분석

### 6-3. 고객군별 제품 카테고리 분석

### 6-4. 월별 제품 카테고리 분석

### 6-5. ARPU (지불유저 1명당 한달에 결제하는 평균 금액)

### 6-6.고객군별 지역 분석석

### 6-7. 고객군별 성별 분포

### 6-8. Q. 마케팅 비용이 고객군별 매출/거래량에 어떤 영향을 끼칠까?

### 6-9. 쿠폰 분석